# COGS 108 - Final Project 

*Fill in your overview here*

# Names

- Mariam Issa
- Andrea Sudharta
- Payam Sadeghian
- Brandon Amaral
- Alex Luo
- Jun Wang

# Group Members IDs

- A12285140
- A########
- A########
- A########
- A########
- A########

# Research Question

*Fill in your research question here*

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis

*Fill in your hypotheses here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
import pandas as pd
import tweepy
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
import requests
import time

In [3]:
tweeters_dem_df = pd.read_csv('popular-tweeters-dem.csv') #Dataframe for list of Democratic Party politicians
tweeters_rep_df = pd.read_csv('popular-tweeters-rep.csv') #Dataframe for list of Republican Party politicians
tweeters_dem_df['Party'] = 'D'
tweeters_rep_df['Party'] = 'R'
tweeters_df = pd.concat([tweeters_dem_df, tweeters_rep_df], ignore_index = True) # Dataframe for combined list of politicians
# print(tweeters_rep_df.shape)
# print(tweeters_rep_df.shape)
# tweeters_df
tweeters_handle = list(tweeters_df['Twitter_Handle']) #Python list for politician Twitter handles
# print(tweeters_handle)

In [4]:
date_intervals = []
request_urls = []

day_delta = datetime.timedelta(days=4)

start_date = datetime.date(2019, 1, 3)
end_date = datetime.date.today()
daterange = (end_date - start_date).days / day_delta.days + 2
for i in range(int(daterange)):
    inc_date = start_date + i*day_delta
    if ((end_date - inc_date).days > 0):
        date_intervals.append(inc_date)
    else:
        date_intervals.append(end_date)
        break

# We want to scrape three days at a time data from January 2019 to May 10, 2019.
# URL format: https://twitter.com/search?l=&q=from%3A{TwitterHandle}%20since%3A{Start date YYYY-MM-DD}%20until%3A{Start date YYY-MM-DD + 4}
for i in tweeters_handle:
    request_url_stub = 'https://twitter.com/search?l=&q=from%3A' + i
    for j in range(len(date_intervals) - 1):
        start_date = date_intervals[j]
        end_date = date_intervals[j+1] - timedelta(days=1)
        request_urls.append(request_url_stub + '%20since%3A' + str(start_date) + '%20until%3A' + str(end_date))
    request_urls.append(request_url_stub + '%20since%3A' + str(date_intervals[-1]) + '%20until%3A' + str(datetime.date.today()))


In [25]:
# Generate Web URL(s) of each congressman
# sampleURL=request_urls[0].replace(" ", "")
# print(sampleURL)
# #Collect Tweets from URL
# page = requests.get(sampleURL)


# all_tweets = []
# soup = BeautifulSoup(page.text, 'html.parser')
# #all_tweets.append(soup.find_all('title'))
# timeline = soup.select('#timeline li.stream-item')
# for i, tweet in enumerate(timeline):
#     print i
#     tweet_id = tweet['data-item-id']
#     tweet_text = tweet.select('p.tweet-text')[0].get_text()
#     all_tweets.append({"id": tweet_id, "text": tweet_text})

https://twitter.com/search?l=&q=from%3A@AOC%20since%3A2019-01-03%20until%3A2019-01-06


In [7]:
# Get OAuth from Twitter
import tweepy

consumer_key = 'bSZRBubFkHWewMi08ltv7DgAu'
consumer_secret = '6JFpZu66GT7OvoJFFxSfJJlX21NH5wxmMjNDbsQGPLyW9WtBSU'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

try:
    redirect_url = auth.get_authorization_url()
    print (redirect_url)
    
except tweepy.TweepError:
    print("Error! Failed to get request token.")

https://api.twitter.com/oauth/authorize?oauth_token=aR-G9QAAAAAA-UgvAAABasqslps


In [8]:
# Verify Twitter account usage
verifier = raw_input('Verifier:')

Verifier:3080815


In [9]:
# Get Access Token
try: 
    auth.get_access_token(verifier)
except tweepy.TweepError:
    print ("Error! Failed to get access token.")

In [26]:
# Save tokens
access_token = auth.access_token
acess_token_secret = auth.access_token_secret

# Note: You do not need to re-fetch it each time. Twitter currently does
# not expire the tokens, so the only time it would ever go invalid is if
# the user revokes our application access. 

#To rebuild an OAuthHandler from the stored access token

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(key, secret)

In [10]:
# Check Set Up
api = tweepy.API(auth)
#api.update_status('tweepy + oauth!') # Posts directly on my timeline

In [11]:
# Filter tweets to only include those containing key words
key_words_by_bill = {
    0:['#savetheinternet', '#netneutrality', 'internet', 'neutrality'], 
    1:['#medicareforall', '#universalhealthcare','#healthcare', 'health', 'care', 'health', 'care', 'medicare'], 
    2: ['#equalpay', '#equalrights', '#equalrightsammendment', '#era', '#women', '#genderinequality', '#genderequality', 'equal', 'rights', 'pay', 'women', 'gender', 'equality'],
    3:['#votingrights', '#votingrightact', '#electionintegrity', '#riggedelections', '#forthepeople', 'voting', 'rights', 'elections', 'fair', 'vote', 'votes'], 
    4:['#guncontrol', '#2ndammendment', '#noguncontol', '#gunviolence', '#gunrights', '#gunreform', '#2a', 'gun', 'second', '2nd',  'amendment'], 
    5:['#yemen', '#yemengenocide', 'yemen', 'yemeni', 'war', 'troops', 'withdraw troops', 'civil'], 
    6:['#women', '#reauthorizationact', '#violenceagainstwomen', '#genderbasedviolence', '#sexualviolence', 'women'],  
    7:['#buildthewall', '#buildthewallnow', '#nationalsecurity', '#nationalemergency', '#trumpswall', '#illegalimmigration', '#mexico', '#border', '#bordercrises', 'mexico', 'illegal', 'immigration', 'security', 'emergency', 'build', 'wall']
}

key_words = list({x for v in key_words_by_bill.itervalues() for x in v})

In [12]:
# Define Intersection Function
def interSection(arr1,arr2): 
    return list(filter(lambda x: x in arr1, arr2)) 

In [13]:
# Collect all tweets that deal with each bill in Congress
bill = [{}] * 8 # bill[bill_number] = {congressman: [tweets about bill]}
tweets = {}

In [15]:
#Iterate through Congressmen
for handle in tweeters_handle:
    all_tweets = []

    for i in range(1000):
        try:
            tweets_from_page_i = api.user_timeline(handle, count = 200, page = i) # max count value is 200
        except:
            print("TweepError: stopped at handle: ", handle, " at page ", i)
        
        # Check if we reached end of Twitter history
        if len(tweets_from_page_i) == 0: 
            break
            
        #iterate through each tweet, sort by bill
        for tweet in tweets_from_page_i:
            tweet_tokens = (tweet.text).lower().split()
            
            # See if Tweet talks about bill matter
            for bill_number, bill_tokens in key_words_by_bill.items():
                if len(interSection(tweet_tokens, bill_tokens)) != 0:
                    if bill[bill_number] == {}:
                        bill[bill_number] = {handle:[tweet.text]}
                    else:
                        if handle not in bill[bill_number]:
                            bill[bill_number][handle] = [tweet.text]
                        else:
                            bill[bill_number][handle] += [tweet.text]
                    
                    # Compile to all tweets list
                    all_tweets.append(tweet.text)
                    
    # Collect congressman/woman's tweets         
    tweets[handle] = all_tweets
        
    # Sleep: to not overload Twitter requests
    time.sleep(45)
    

#Create look up table for Congressman/woman's name and party
tweeters_party = list(tweeters_df['Party'])
tweeters_name = list(tweeters_df['Name'])

handle_to_party = {}
handle_to_name = {}
for i, handle in enumerate(tweeters_handle):
    handle_to_party[handle] = tweeters_party[i]
    handle_to_name[handle] = tweeters_name[i]

data = [] 

# List all tweets by bill it's associated with and the congressman who tweeted it
for i in range(len(bill)):
    for handle, tweet in bill[i].items():
        data.append([handle_to_name[handle], handle_to_party[handle], handle, bill_title[i], tweet])
        
tweeters_df = pd.DataFrame(data, columns = ['Name', 'Party', 'Twitter Handle', 'Bill #', 'Tweet'])

# Export CSV with Tweets
tweeters_df.to_csv('tweets_by_bill')

('TweepError: stopped at handle: ', '@RobbWittman ', ' at page ', 0)


In [14]:
bill_title = {
    0: "Save the Internet",
    1: "Health Care",
    2: "Paycheck Fairness Act",
    3: "For the People Act 2019",
    4: "Background Check for Firearms",
    5: "Remove Troops from Yemen",
    6: "Violence against Women Reauthorization Act",
    7: "Trump's National Emergency"
}

In [151]:
#Create look up table for Congressman/woman's name and party
tweeters_party = list(tweeters_df['Party'])
tweeters_name = list(tweeters_df['Name'])

handle_to_party = {}
handle_to_name = {}
for i, handle in enumerate(tweeters_handle):
    handle_to_party[handle] = tweeters_party[i]
    handle_to_name[handle] = tweeters_name[i]

data = [] 

# List all tweets by bill it's associated with and the congressman who tweeted it
for i in range(len(bill)):
    for handle, tweet in bill[i].items():
        data.append([handle_to_name[handle], handle_to_party[handle], handle, bill_title[i], tweet])
        
tweeters_df = pd.DataFrame(data, columns = ['Name', 'Party', 'Twitter Handle', 'Bill #', 'Tweet'])

# Export CSV with Tweets
tweeters_df.to_csv('tweets_by_bill')

In [157]:
# Export CSV with Tweets
for handle, tweet_list in tweets.items():
    for tweet in tweet_list:
        tweet = tweet.encode('utf-8')
        
tweets_df = pd.DataFrame.from_dict(tweets, orient='index')
tweets_df.to_csv('congressmember_tweets')

UnicodeEncodeError: 'ascii' codec can't encode character u'\u2026' in position 114: ordinal not in range(128)

In [158]:
# Collect all, unfiltered tweets from each Congressman/Woman
dem_tweets = {} # key = Congressman/woman's Twitter Handle, value = [<tweets>]
rep_tweets = {}

# List Congressman's Party
tweeter_party = list(tweeters_df['Party'])

#Iterate through Congressmen
for handle, party in zip(tweeters_handle, tweeter_party):
    # Added since tweet gathering stalled
    if (handle in dem_tweets) or (handle in rep_tweets):
        continue
        
    all_tweets = []
    for i in range(1000):
        tweets_from_page_i = api.user_timeline(handle, count = 200, page = i) # max count value is 200
        
        # Check if we compiled tweets from all pages
        if len(tweets_from_page_i) == 0:
            break
            
        # Add all tweets from page i
        all_tweets.extend([t.text for t in tweets_from_page_i])
    
    if party == 'D':
        dem_tweets[handle] = all_tweets
    else:
        rep_tweets[handle] = all_tweets   
    time.sleep(45)

TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out.

In [122]:
# Create Dataframe from each Dictionary (unfiltered tweets) --> CSV File

# Democrats
dem_tweets_df = pd.DataFrame.from_dict(dem_tweets, orient='index')
dem_tweets_df.to_csv('dem_tweets_unfiltered')

# Republicans
rep_tweets_df = pd.DataFrame.from_dict(rep_tweets, orient='index')
rep_tweets_df.to_csv('rep_tweets_unfiltered')

UnicodeEncodeError: 'ascii' codec can't encode character u'\u2026' in position 139: ordinal not in range(128)

# Data Cleaning

Describe your data cleaning steps here.

In [4]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*